In [1]:
import psycopg2
import authentication as auth
import pandas as pd


import numpy as np
import datetime

import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score



In [2]:
conn = psycopg2.connect(database=auth.database, user=auth.userid, password=auth.password, host=auth.host)
cur = conn.cursor()

In [76]:
#df = pd.read_sql_query("SELECT cme.rcra_id, cme.start_date, cme.evaluation_count, cme.violation, cme.enforcement, cme.formal_enforcement, mani.approx_qty, mani.waste_code_1  FROM summary.cmecomp_reduced cme INNER JOIN raw.mani06_16 mani ON (mani.gen_rcra_id = cme.rcra_id);", conn, parse_dates= ['start_date'])

df = pd.read_sql_query("SELECT cme.rcra_id, cme.start_date, cme.evaluation_count, cme.violation, cme.enforcement, cme.formal_enforcement FROM summary.cmecomp_reduced cme", conn, parse_dates= ['start_date'])



In [79]:
df.dtypes

rcra_id                       object
start_date            datetime64[ns]
evaluation_count               int64
violation                       bool
enforcement                     bool
formal_enforcement              bool
start_year                     int64
dtype: object

In [80]:
#fix variable types
df['violation'].astype(int).head()
df['start_year'] = df['start_date'].dt.year
#df['waste_code'] = df.waste_code_1.str[0:1]
df['state'] = df.rcra_id.str[0:2]
df['violation_dum'] = (df.violation == True).astype(int)
df['enforcement_dum'] = (df.enforcement == True).astype(int)
df['formal_enforcement'] = (df.formal_enforcement == True).astype(int)
df['start_year'] = df.start_year.astype(str)

In [81]:
df.head()

,rcra_id,start_date,evaluation_count,violation,enforcement,formal_enforcement,start_year,state,violation_dum,enforcement_dum
0,NY0000000950,1996-04-25,1,False,False,0,1996,NY,0,0
1,NY0000001107,2009-03-24,28,True,True,1,2009,NY,1,1
2,NY0000001107,2009-09-10,28,True,True,1,2009,NY,1,1
3,NY0000001107,2010-05-20,1,False,False,0,2010,NY,0,0
4,NY0000001107,2010-08-26,5,True,True,0,2010,NY,1,1


In [75]:
test = df.drop_duplicates(['rcra_id','start_year'], keep = 'first')

KeyError: 'start_year'

In [10]:
#explore some crosstabs

print(pd.crosstab(df['waste_code'], df['enforcement_dum'], rownames=['waste_code']))

enforcement_dum        0        1
waste_code                       
                  104129    71472
B                 746796   366755
D                6886287  4178530
F                1383496  1271687
K                  62282    43181
P                  80841    60217
S                      8       20
U                 206485   147559


In [11]:
print(pd.crosstab(df['waste_code'], df['violation_dum'], rownames=['waste_code']))

violation_dum        0        1
waste_code                     
                101149    74452
B               700928   412623
D              6600487  4464330
F              1298877  1356306
K                57113    48350
P                75600    65458
S                    8       20
U               195564   158480


In [12]:
print(pd.crosstab(df['state'], df['violation_dum'], rownames=['state']))

violation_dum        0        1
state                          
NY             9044996  6592957


In [13]:
print(pd.crosstab(df['start_year'], df['violation_dum'], rownames=['start_year']))

violation_dum       0       1
start_year                   
1981              115    2710
1982               48     371
1983            29904   51663
1984           339473  168300
1985           205820  185737
1986           367362  225615
1987           440681  412304
1988           330556  272838
1989           209806  115477
1990           146487  160977
1991           251706  133766
1992           152293  119145
1993           176449  167894
1994           178590  127658
1995           215827   94441
1996           140671  225186
1997           256671  120977
1998           173018  150504
1999           343114  174432
2000           325905  158080
2001           356938  206833
2002           261428  152096
2003           341973  262442
2004           407300  171925
2005           347779  356944
2006           293520  268034
2007           180492  248988
2008           343411  380786
2009           365362  229129
2010           264974  185371
2011           303464  222221
2012      

# Subset to 2006-2015



In [21]:
#set begin and end dates for mask
begin_date= pd.to_datetime('2006-01-01', format = '%Y-%m-%d')
begin_date

end_date= pd.to_datetime('2015-12-31', format = '%Y-%m-%d')
end_date

Timestamp('2015-12-31 00:00:00')

In [22]:
mask = (df['start_date'] > begin_date) & (df['start_date'] <= end_date)

In [23]:
#subset
dfsub = df.loc[mask]

In [60]:
##################3
#Sanity check - subset
dfsub = dfsub.sort(['approx_qty'], ascending=[1]).head(500)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [61]:
#check
np.unique(dfsub.start_year)

array(['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015'], 
      dtype='<U4')

In [62]:
#dummify years and waste_codes
dummy_years = pd.get_dummies(dfsub['start_year'], prefix = 'year').astype(np.int64)
print(dummy_years.head())
dummy_waste_codes = pd.get_dummies(dfsub['waste_code'], prefix = 'waste_code').astype(np.int64)


          year_2006  year_2007  year_2008  year_2009  year_2010  year_2011  \
15263127          0          0          0          0          0          0   
15262625          1          0          0          0          0          0   
15262876          0          0          0          1          0          0   
15262872          0          0          0          1          0          0   
15263123          0          0          0          0          0          0   

          year_2012  year_2013  year_2014  year_2015  
15263127          0          0          1          0  
15262625          0          0          0          0  
15262876          0          0          0          0  
15262872          0          0          0          0  
15263123          0          0          1          0  


In [63]:
#set features for baby model & add dummies to existing dataframe (dfsub)
selectVars = ['violation_dum','approx_qty']

#modelData = dfsub[selectVars].join(dummy_years.ix[:, 'year_2007':]).join(dummy_waste_codes.ix[:, 'waste_code_B':])
modelData = dfsub[selectVars]

In [64]:
modelData.head()

,violation_dum,approx_qty
15263127,1,0.000220
15262625,1,0.000220
15262876,1,0.000220
15262872,1,0.000287
15263123,1,0.000287


In [65]:
#add an intercept manually
modelData['intercept'] = 1.0

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [66]:
#select features dataframe to be everything except the response
features = modelData.columns[1:]
features

Index(['approx_qty', 'intercept'], dtype='object')

In [67]:
#run logit
logit1 = sm.Logit(modelData['violation_dum'], modelData[features])
result1 = logit1.fit()

Optimization terminated successfully.
         Current function value: 0.609636
         Iterations 28


In [68]:
#explore results
print(result1.summary())

                           Logit Regression Results                           
Dep. Variable:          violation_dum   No. Observations:                  500
Model:                          Logit   Df Residuals:                      498
Method:                           MLE   Df Model:                            1
Date:                Mon, 11 Jul 2016   Pseudo R-squ.:                 0.02513
Time:                        04:48:56   Log-Likelihood:                -304.82
converged:                       True   LL-Null:                       -312.68
                                        LLR p-value:                 7.357e-05
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
approx_qty -3553.2457   2297.824     -1.546      0.122     -8056.898   950.407
intercept      7.0209      5.060      1.388      0.165        -2.896    16.938
